<a href="https://colab.research.google.com/github/Nithish1010/Youtube_Transcript_Summarizer_Hugging_Face_Speech-to-Text_ASR/blob/main/YTSASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install pytube -q

In [34]:
from pytube import YouTube

In [35]:
Video_Url="https://www.youtube.com/watch?v=1aA1WGON49E"

In [36]:
yt=YouTube(Video_Url)

In [38]:
yt.streams\
  .filter(only_audio=True,file_extension='mp4') \
  .first()\
  .download(filename = 'ytaudio.mp4' )

'/content/ytaudio.mp4'

In [40]:
! ffmpeg -i ytaudio.mp4 -acodec pcm_s16le -ar 16000 ytaudio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

**English ASR WITH HUGGING SOUND**

In [32]:
!pip install huggingsound -q

In [41]:
from huggingsound import SpeechRecognitionModel

In [42]:
import torch
device="cuda" if torch.cuda.is_available() else "cpu"

In [43]:
model=SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english",device=device)

INFO:huggingsound.speech_recognition.model:Loading model...


**Audio chunking**

In [44]:
import librosa

In [45]:
input_file='/content/ytaudio.wav'

In [46]:
print(librosa.get_samplerate(input_file))

stream=librosa.stream(
    input_file,
    block_length=30,
    frame_length=1600,
    hop_length=16000
)

16000


In [47]:
import soundfile as sf

In [48]:
for i, speech in enumerate(stream):
  sf.write(f'{i}.wav',speech,16000)

**CHUNK TRANSCRIPTION**

In [49]:
audio_path=[]
for a in range (i+1):
  audio_path.append(f'/content/{a}.wav')

In [50]:
audio_path

['/content/0.wav',
 '/content/1.wav',
 '/content/2.wav',
 '/content/3.wav',
 '/content/4.wav',
 '/content/5.wav',
 '/content/6.wav',
 '/content/7.wav',
 '/content/8.wav',
 '/content/9.wav',
 '/content/10.wav',
 '/content/11.wav',
 '/content/12.wav',
 '/content/13.wav',
 '/content/14.wav',
 '/content/15.wav',
 '/content/16.wav',
 '/content/17.wav',
 '/content/18.wav',
 '/content/19.wav',
 '/content/20.wav',
 '/content/21.wav',
 '/content/22.wav',
 '/content/23.wav',
 '/content/24.wav',
 '/content/25.wav',
 '/content/26.wav',
 '/content/27.wav',
 '/content/28.wav',
 '/content/29.wav',
 '/content/30.wav',
 '/content/31.wav',
 '/content/32.wav',
 '/content/33.wav',
 '/content/34.wav',
 '/content/35.wav',
 '/content/36.wav',
 '/content/37.wav',
 '/content/38.wav',
 '/content/39.wav',
 '/content/40.wav',
 '/content/41.wav',
 '/content/42.wav',
 '/content/43.wav',
 '/content/44.wav',
 '/content/45.wav',
 '/content/46.wav',
 '/content/47.wav',
 '/content/48.wav',
 '/content/49.wav',
 '/content

In [51]:
transcriptions=model.transcribe(audio_path)

100%|██████████| 59/59 [00:26<00:00,  2.21it/s]


In [52]:
full_transcript=''

In [53]:
for item in transcriptions:
  full_transcript += ''.join(item['transcription'])

In [54]:
full_transcript

"whyl what an audiencebut ifbeing honest i don't care what you think  y talkdont i care what the internet thinks might talkthey re the ones wou get it seen and get it shared and i think that's where most people get wrong they're talking to you here stafwhyl what an audiencebut ifbeing honest i don't care what you think  my alki dont i care what the internet thinks might talkthey are the ones wou get it seen and get it shared and i think that's where most people get wrong they're talking to you here stead of talking to youwhyl what an audiencebut fbeing honest i don't care what you think  my alki dont i care what the internet thinks might talkthey re the ones wou get it seen and get it shared and i think that's where most people get wrong they're talking to you here stead of talking to you randomwhil what an audiencebut ifbeing honest i don't care what you think o m alk dont i care what the internet thinks might talk they re the ones wou get it seen and get it shared and i think that's 

TEXT SUMMARIZATION


In [55]:
from transformers import pipeline

In [56]:
summarization = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [58]:
num_iters=int(len(full_transcript)/1000)
summarized_text=[]
for i in range(0,num_iters+1):
  start=0
  start=i*1000
  end=(i+1)*1000
  out=summarization(full_transcript[start:end],min_length=5,max_length=20)
  out=out[0]
  out=out['summary_text']
  summarized_text.append(out)

In [60]:
print(summarized_text)

[" i think that's where most people get wrong they're talking to you here staf", " i think that's where most people get wrong they're talking to you here stead of", " i don't care what you think of my alkonti care what the internet thinks", " i don't care what you think of my alkonti care what the internet thinks", ' Back in two-thousand nine we all have these weard little things called attent', ' Back in two-thousand nine we all have these weard little things called attent', " i think that's where most people get awrong they're talking to you here", ' We all have these weard little things called attention spansye theyre theyre we', " i think that's where most people get awrong they're talking to you here", ' Back in two-thousand nine we all have these weard little things called attention', ' Back in two-thousand nine we all had these weard little things called attention', " i'm trying to think of the last time i watched an eighteen minutes kt's", " i'm trying to think of the last time